[Node 18: Synchronisation](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node18.html)

Navigation:

**Next:** [Python-Threads und GIL](node19.ipynb) **Up:** [Python-Threads und GIL](node19.ipynb) **Previous:** [Python-Threads und GIL](node19.ipynb)

## synchronization
In the previous examples, the threads run independently of each other. It gets more complicated when they access shared data areas, especially when one thread is writing and another is reading. The following – somewhat constructed – example illustrates the problem.


In [ ]:
import threading 
  
class Counter(object):
    def __init__(self):
        self.num = 0
    def inc(self):
        self.num += 1

def thread_task(counter):
    for _ in range(100000):
        counter.inc()
  
# global counter
counter = Counter()

# create two threads 
t1 = threading.Thread(target = thread_task, args = (counter,))
t2 = threading.Thread(target = thread_task, args = (counter,))
  
# start threads 
t1.start() 
t2.start() 
  
# wait until threads finish their job 
t1.join() 
t2.join() 
  
# print result
print(counter.num) 


**What do you get out**?

Both threads access the same <font color=#008000> *Counter object*</font>, i.e. they use the same <font color=#ff0000> **member variable**</font> **self.num* *.
 

Depending on the randomness, we get the expected result (200000) or another number between 100000 and 200000. The program is non-deterministic, although no random numbers are used... This phenomenon is called ["race condition"](https://dribbble.com/shots/3157258-Race-Condition-Programming-Joke) because the dependence of the result on the call order, which in turn depends on the distribution of CPU time among each thread.
 

In the example, you can define/use the variables differently and thus solve the specific problem, e.g. give each thread its own counter. But this is not a solution for the general case.

## locking
The <font color=#ff0000> **Locking**</font> mechanism offers a real remedy in Python (among others):
* <font color=#0000e6> ``lock = threading.Lock()``</font> object is created
* Calling <font color=#0000e6> ``lock.acquire()``</font> before executing critical area
* First thread that calls <font color=#0000e6> ``lock.acquire()``</font> keeps running
* Next thread that <font color=#0000e6> ``lock.acquire()``</font> has to wait
* until first thread <font color=#0000e6> calls ``lock.release()``</font>
* etc.

This ensures that the <font color=#008000> *inc()*</font> method is only executed by <font color=#ff0000> **one**</font> thread at a time, all others are as long as blocked.

In [ ]:
import threading 
  
class Counter(object):
    def __init__(self):
        self.num = 0
    def inc(self):
        self.num += 1

def thread_task(counter, lock):
    for _ in range(100000):
        lock.acquire()
        counter.inc()
        lock.release()
  
# global counter
counter = Counter()

# lock
lock    = threading.Lock() 

# create two threads 
t1 = threading.Thread(target = thread_task, args = (counter,lock))
t2 = threading.Thread(target = thread_task, args = (counter,lock))
  
# start threads 
t1.start() 
t2.start() 
  
# wait until threads finish their job 
t1.join() 
t2.join() 
  
# print result
print(counter.num) 


Now the result is correct, but the `locking` leads to speed losses.

**Task**: Think about how you rate the relative execution speed of the wrong and right variant, but also compared to a single-threaded variant. Measure the speed of the different options (hint: `%%time`) and compare them. Explain the result.

In this case, the two threads together will even be significantly slower than a single one. Programming with parallel threads obviously needs great care and planning to really bring about a speed gain.